# Importa módulos necessários
## O módulo `abc` é utilizado para definir classes abstratas. "ABC" é a sigla para "Abstract Base Classes".

In [1]:
from abc import ABC

## Importa os módulos do QT para desenhar gráficos

In [2]:
from PySide6.QtCore import Qt, QPointF
from PySide6.QtGui import QPolygonF
from PySide6.QtWidgets import QApplication, QGraphicsView, QGraphicsScene, QGraphicsEllipseItem, QGraphicsLineItem, QGraphicsPolygonItem

## O módulo `xml.etree.ElementTree` é utilizado para manipular arquivos XML e é usado como parser

In [3]:
import xml.etree.ElementTree as ElementTree

## O módulo sys é usado para passar argumentos de linha de comando para a instância QApplication e para garantir uma saída limpa.

In [4]:
import sys

# Definição das classes e métodos

## Define a classe "Ponto", que representa um ponto em um epaço bidimensional com coordenadas x e y. Inclui métodos para comparar pontos e imprimir suas coordenadas.

In [5]:
class Ponto:
    def __init__(self, x: float, y: float) -> None:
        self._x = float(x)
        self._y = float(y)

    @property
    def x(self) -> float:
        return self._x

    @property
    def y(self) -> float:
        return self._y

    def __eq__(self, __value: object) -> bool:
        return self.x == __value.x and self.y == __value.y

    def print(self) -> None:
        print("x: " + str(self._x), "y: " + str(self._y), sep=", ")

## Define a classe "Retangulo", que representa um retângulo com vértices em seus cantos. Inclui métodos para calcular a largura e a altura do retângulo e para imprimir suas coordenadas.

In [6]:
class Retangulo(ABC):
    def __init__(self, p_minimo: Ponto, p_maximo: Ponto) -> None:
        if not all(isinstance(p, Ponto) for p in (p_minimo, p_maximo)):
            raise TypeError("As coordenadas devem ser um Ponto")
        self._p_minimo = p_minimo
        self._p_maximo = p_maximo

    @property
    def p_minimo(self) -> Ponto:
        return self._p_minimo

    @property
    def p_maximo(self) -> Ponto:
        return self._p_maximo

    def comprimento(self) -> float:
        return self._p_maximo.x - self._p_minimo.x

    def altura(self) -> float:
        return self._p_maximo.y - self._p_minimo.y

    def print(self) -> None:
        print("Ponto mínimo:", end=" ")
        self._p_minimo.print()
        print("Ponto máximo:", end=" ")
        self._p_maximo.print()
        print("Dimensão:", end=" ")
        print(self.comprimento(), self.altura(), sep="x")

## Define as classes "Window" e "Viewport", que herdam de "Retangulo". A classe "Window" representa a janela de visualização e a classe "Viewport" representa a área de visualização.

In [7]:
class Window(Retangulo):
    def print(self) -> None:
        print("---- Window ----")
        super().print()

class Viewport(Retangulo):
    def print(self) -> None:
        print("---- Viewport ----")
        super().print()

## Define a classe "Reta", que representa uma reta com dois pontos. Inclui métodos para calcular o tamanho da reta e para imprimir suas coordenadas.

In [8]:
class Reta:
    def __init__(self, a: Ponto, b: Ponto) -> None:
        if not all(isinstance(p, Ponto) for p in (a, b)):
            raise TypeError("As coordenadas devem ser um Ponto")
        if a == b:
            raise ValueError("Pontos da reta não podem coincidir")
        self._a = a
        self._b = b

    @property
    def a(self) -> Ponto:
        return self._a

    @property
    def b(self) -> Ponto:
        return self._b

    def tamanho(self) -> float:
        dx = self._b.x - self._a.x
        dy = self._b.y - self._a.y
        return (dx**2 + dy**2)**0.5

    def print(self) -> None:
        print("---- Reta ----")
        self._a.print()
        self._b.print()
        print("Tamanho: ", self.tamanho())

## Define a classe "Poligono", que representa um polígono com uma lista de pontos. Inclui métodos para imprimir as coordenadas do polígono.

In [9]:
class Poligono:
    def __init__(self, *pontos: Ponto) -> None:
        if not all(isinstance(p, Ponto) for p in pontos):
            raise TypeError("As coordenadas devem ser um Ponto")
        if len(pontos) <= 2:
            raise ValueError("Quantidade de pontos do poligono deve ser maior que 2")
        self._pontos = list(pontos)

    @property
    def pontos(self) -> list[Ponto]:
        return self._pontos

    def print(self) -> None:
        print("---- Poligono ----")
        for ponto in self._pontos:
            ponto.print()

## Define a classe "ViewportWindow", que herda de "QGraphicsView". A classe é utilizada para criar uma janela de visualização com uma cena gráfica, onde são adicionados pontos, retas e polígonos.

In [10]:
class ViewportWindow(QGraphicsView):
    def __init__(self, viewport, pontos, retas, poligonos):
        super().__init__()
        self.setWindowTitle("Viewport")
        self.setBackgroundBrush(Qt.white)

        # Criar a dimensão da viewport
        self.setSceneRect(viewport.p_minimo.x, viewport.p_minimo.y, viewport.p_maximo.x, viewport.p_maximo.y)

        # Criar uma cena gráfica
        scene = QGraphicsScene()
        self.setScene(scene)

        # Adicionar pontos
        for ponto in pontos:
            ponto_vp = QGraphicsEllipseItem(ponto.x, ponto.y, 1, 1)
            scene.addItem(ponto_vp)

        # Adicionar retas
        for reta in retas:
            reta_vp = QGraphicsLineItem(reta.a.x, reta.a.y, reta.b.x, reta.b.y)
            scene.addItem(reta_vp)

        # Adicionar polígonos
        for poligono in poligonos:
            poligono_vp = QGraphicsPolygonItem()
            poligono_vp.setPolygon(QPolygonF([QPointF(ponto.x, ponto.y) for ponto in poligono.pontos]))
            scene.addItem(poligono_vp)

## Define a função "transformar_pontos_viewport", que recebe uma janela, uma viewport e uma lista de pontos e retorna uma lista de pontos transformados para a viewport.

In [11]:
def transformar_pontos_viewport(window: Window, viewport: Viewport, pontos: list[Ponto]) -> list[Ponto]:
    pontos_vp = []
    for ponto in pontos:
        x_vp = ((ponto.x - window.p_minimo.x) / window.comprimento()) * viewport.comprimento()
        y_vp = (1 - ((ponto.y - window.p_minimo.y) / window.altura())) * viewport.altura()
        pontos_vp.append(Ponto(x_vp, y_vp))
    return pontos_vp

# Main

## Lê o arquivo XML e armazena os dados em uma variável

In [12]:
dados = ElementTree.parse('docs/entrada.xml').getroot()

## Lê os pontos da window arquivo XML e constroi a window

In [13]:
w_min = Ponto(*dados.find("window/wmin").attrib.values())
w_max = Ponto(*dados.find("window/wmax").attrib.values())
window = Window(w_min, w_max)
window.print()

---- Window ----
Ponto mínimo: x: 0.0, y: 0.0
Ponto máximo: x: 10.0, y: 10.0
Dimensão: 10.0x10.0


## Lê os pontos da viewport arquivo XML e constroi a viewport

In [14]:
vp_min = Ponto(*dados.find("viewport/vpmin").attrib.values())
vp_max = Ponto(*dados.find("viewport/vpmax").attrib.values())
viewport = Viewport(vp_min, vp_max)
viewport.print()

---- Viewport ----
Ponto mínimo: x: 10.0, y: 10.0
Ponto máximo: x: 630.0, y: 470.0
Dimensão: 620.0x460.0


## Lê os pontos do arquivo XML, constrói os pontos e os imprime

In [15]:
pontos = [Ponto(*ponto.attrib.values()) for ponto in dados.findall("ponto")]
for ponto in pontos:
    ponto.print()

x: 0.0, y: 2.0
x: 2.0, y: 4.0
x: 4.0, y: 6.0


## Lê os pontos das retas do arquivo XML, constroi as retas e as imprime

In [16]:
retas = [
    Reta(*(Ponto(*ponto.attrib.values()) for ponto in reta))
    for reta in dados.findall("reta")
]
for reta in retas:
    reta.print()

---- Reta ----
x: 2.0, y: 2.0
x: 4.0, y: 4.0
Tamanho:  2.8284271247461903
---- Reta ----
x: 4.0, y: 4.0
x: 2.0, y: 6.0
Tamanho:  2.8284271247461903


## Lê os pontos dos polígonos do arquivo XML, constroi os polígonos e imprime

In [17]:
poligonos = [
    Poligono(*(Ponto(*ponto.attrib.values()) for ponto in poligono))
    for poligono in dados.findall("poligono")
]
for poligono in poligonos:
    poligono.print()

---- Poligono ----
x: 1.0, y: 1.0
x: 1.0, y: 2.0
x: 2.0, y: 2.0
x: 2.0, y: 1.0


## Limpa os dados da estrutura antiga, que já foi processada.

In [18]:
dados.clear()

## Transforma os pontos para a viewport e imprime as coordenadas dos pontos transformados

In [19]:
pontos = transformar_pontos_viewport(window, viewport, pontos)
for ponto in pontos:
    ponto_elem = ElementTree.SubElement(dados, 'ponto')
    ponto_elem.set("x", str(ponto.x))
    ponto_elem.set("y", str(ponto.y))
    ponto.print()

x: 0.0, y: 368.0
x: 124.0, y: 276.0
x: 248.0, y: 184.0


## Transforma os pontos das retas para a viewport e imprime as coordenadas dos pontos transformados

In [20]:
retas = [Reta(*transformar_pontos_viewport(window, viewport, [reta.a, reta.b])) for reta in retas]
for reta in retas:
    reta_elem = ElementTree.SubElement(dados, "reta")
    for ponto in [reta.a, reta.b]:
        ponto_elem = ElementTree.SubElement(reta_elem, "ponto")
        ponto_elem.set("x", str(ponto.x))
        ponto_elem.set("y", str(ponto.x))
    reta.print()

---- Reta ----
x: 124.0, y: 368.0
x: 248.0, y: 276.0
Tamanho:  154.40207252495026
---- Reta ----
x: 248.0, y: 276.0
x: 124.0, y: 184.0
Tamanho:  154.40207252495026


## Transforma os pontos dos polígonos para a viewport e imprime as coordenadas dos pontos transformados

In [21]:
poligonos = [Poligono(*transformar_pontos_viewport(window, viewport, poligono.pontos)) for poligono in poligonos]
for poligono in poligonos:
    poligono_elem = ElementTree.SubElement(dados, "poligono")
    for ponto in poligono.pontos:
        ponto_elem = ElementTree.SubElement(poligono_elem, "ponto")
        ponto_elem.set("x", str(ponto.x))
        ponto_elem.set("y", str(ponto.x))
    poligono.print()

---- Poligono ----
x: 62.0, y: 414.0
x: 62.0, y: 368.0
x: 124.0, y: 368.0
x: 124.0, y: 414.0


## Salva o arquivo de saída com a nova estrutura

In [22]:
xml = ElementTree.ElementTree(dados)
ElementTree.indent(xml, space="\t", level=0)
xml.write("docs/saida.xml", xml_declaration=True, encoding="utf-8")

## Gera o viewport

In [23]:
app = QApplication(sys.argv)
viewport_window = ViewportWindow(viewport, pontos, retas, poligonos)
viewport_window.show()
sys.exit(app.exec())

SystemExit: 0

C:\Users\Xande\PycharmProjects\tp1cg\venv\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
